# Sudoku-6 : Resolution par CSP Academique (AIMA)

**Niveau** : Programmation par Contraintes
**Duree** : ~25 min
**Prerequis** : Sudoku-0-Environment

## Objectifs d'apprentissage

1. **Formaliser** le Sudoku comme un CSP (variables, domaines, contraintes)
2. **Implementer** les algorithmes de reference AIMA : AC-3, Forward Checking, MAC
3. **Appliquer** les heuristiques MRV et LCV pour optimiser la recherche
4. **Comparer** experimentalement les differentes strategies de resolution

## Navigation

| << Precedent | [Index](./README.md) | Suivant >> |
|-------------|---------------------|-----------|
| [Sudoku-5-PSO](Sudoku-5-PSO.ipynb) | | [Sudoku-7-Norvig](Sudoku-7-Norvig.ipynb) |

## 1. Introduction : Le cadre AIMA

Ce notebook presente la resolution de Sudoku selon l'approche academique decrite dans **"Artificial Intelligence: A Modern Approach"** (Russell & Norvig, 4e edition, Chapitre 6).

### Pourquoi cette approche ?

Contrairement aux bibliothèques industrielles (OR-Tools, Choco) ou aux metaheuristiques (GA, SA, PSO), l'approche AIMA :
- **Est pedagogique** : chaque composant est transparent et comprehensible
- **Est modulaire** : on peut combiner differentes heuristiques et propagations
- **Sert de reference** : c'est le standard academique pour comparer les algorithmes

### Formalisation CSP du Sudoku

| Composant | Description | Taille
|-----------|-------------|-------
| **Variables** | $X_{i,j}$ pour chaque cellule (i, j) | 81
| **Domaines** | $D_{i,j} \subseteq \{1, 2, \ldots, 9\}$ | 9 valeurs max
| **Contraintes** | AllDifferent par ligne, colonne, bloc 3x3 | 27 contraintes

### Algorithmes couverts

| Algorithme | Type | Complexite | Puissance
|------------|------|------------|----------
| **Backtracking simple** | Recherche | $O(d^n)$ | Faible
| **Backtracking + MRV** | Heuristique | Variable | Moderee
| **Forward Checking** | Propagation 1-niveau | $O(n \cdot d^2)$ | Moderee
| **AC-3** | Arc-consistance | $O(e \cdot d^3)$ | Forte
| **MAC** | AC-3 + Backtracking | Variable | Tres forte

In [ ]:
#!import Sudoku-0-Environment-Csharp.ipynb

## 2. Classe CSP generique

Nous definissons une classe CSP generique inspiree du livre AIMA. Cette classe represente un **CSP binaire** (contraintes entre paires de variables).

In [ ]:
using System.Collections.Generic;
using System.Linq;

/// <summary>
/// Probleme de Satisfaction de Contraintes (CSP) binaire.
/// Inspire de AIMA - Russell & Norvig, Chapitre 6.
/// </summary>
public class CSP<TVariable, TValue> where TVariable : notnull
{
    public List<TVariable> Variables { get; }
    public Dictionary<TVariable, List<TValue>> Domains { get; }
    public Dictionary<TVariable, List<TVariable>> Neighbors { get; }
    public Func<TVariable, TValue, TVariable, TValue, bool> ConstraintFunc { get; }
    
    // Compteurs pour l'analyse
    public int NumAssignments { get; set; }
    public int NumBacktracks { get; set; }

    public CSP(
        IEnumerable<TVariable> variables,
        Dictionary<TVariable, IEnumerable<TValue>> domains,
        Dictionary<TVariable, IEnumerable<TVariable>> neighbors,
        Func<TVariable, TValue, TVariable, TValue, bool> constraintFunc)
    {
        Variables = variables.ToList();
        Domains = domains.ToDictionary(kvp => kvp.Key, kvp => kvp.Value.ToList());
        Neighbors = neighbors.ToDictionary(kvp => kvp.Key, kvp => kvp.Value.ToList());
        ConstraintFunc = constraintFunc;
        NumAssignments = 0;
        NumBacktracks = 0;
    }

    /// <summary>
    /// Verifie si (var, val) est consistant avec l'assignation partielle.
    /// </summary>
    public bool IsConsistent(TVariable var, TValue val, Dictionary<TVariable, TValue> assignment)
    {
        foreach (var neighbor in Neighbors[var])
        {
            if (assignment.TryGetValue(neighbor, out var neighborValue))
            {
                if (!ConstraintFunc(var, val, neighbor, neighborValue))
                    return false;
            }
        }
        return true;
    }

    /// <summary>
    /// Verifie si l'assignation est complete.
    /// </summary>
    public bool IsComplete(Dictionary<TVariable, TValue> assignment)
    {
        return assignment.Count == Variables.Count;
    }

    /// <summary>
    /// Retourne une copie profonde des domaines.
    /// </summary>
    public Dictionary<TVariable, List<TValue>> CopyDomains()
    {
        return Domains.ToDictionary(kvp => kvp.Key, kvp => kvp.Value.ToList());
    }

    /// <summary>
    /// Retourne tous les arcs (Xi, Xj) du CSP.
    /// </summary>
    public List<(TVariable, TVariable)> GetArcs()
    {
        var arcs = new List<(TVariable, TVariable)>();
        foreach (var var in Variables)
            foreach (var neighbor in Neighbors[var])
                arcs.Add((var, neighbor));
        return arcs;
    }
}

Console.WriteLine("Classe CSP<TVariable, TValue> definie.");

## 3. Construction du CSP Sudoku

Nous transformons une grille Sudoku en instance CSP avec :
- **81 variables** : une par cellule (0,0) a (8,8)
- **Domaines** : {1..9} pour les cellules vides, {v} pour les cellules fixees
- **Contraintes** : AllDifferent representee comme paires binaires != 

In [ ]:
/// <summary>
/// Construit un CSP a partir d'une grille Sudoku.
/// </summary>
public static class SudokuCSPBuilder
{
    public static CSP<(int, int), int> BuildCSP(SudokuGrid grid)
    {
        // Variables : (row, col) pour chaque cellule
        var variables = new List<(int, int)>();
        for (int i = 0; i < 9; i++)
            for (int j = 0; j < 9; j++)
                variables.Add((i, j));

        // Domaines : 1-9 pour les vides, valeur unique pour les fixees
        var domains = new Dictionary<(int, int), IEnumerable<int>>();
        for (int i = 0; i < 9; i++)
        {
            for (int j = 0; j < 9; j++)
            {
                var value = grid.Cells[i][j];
                if (value == 0)
                    domains[(i, j)] = Enumerable.Range(1, 9);
                else
                    domains[(i, j)] = new[] { value };
            }
        }

        // Voisins : meme ligne, meme colonne, meme bloc
        var neighbors = new Dictionary<(int, int), IEnumerable<(int, int)>>();
        foreach (var (i, j) in variables)
        {
            var neighborSet = new HashSet<(int, int)>();
            
            // Meme ligne
            for (int k = 0; k < 9; k++)
                if (k != j) neighborSet.Add((i, k));
            
            // Meme colonne
            for (int k = 0; k < 9; k++)
                if (k != i) neighborSet.Add((k, j));
            
            // Meme bloc 3x3
            int blockRow = (i / 3) * 3;
            int blockCol = (j / 3) * 3;
            for (int r = blockRow; r < blockRow + 3; r++)
                for (int c = blockCol; c < blockCol + 3; c++)
                    if (r != i || c != j) neighborSet.Add((r, c));
            
            neighbors[(i, j)] = neighborSet;
        }

        // Fonction de contrainte : valeurs differentes
        Func<(int, int), int, (int, int), int, bool> constraint = (v1, val1, v2, val2) => val1 != val2;

        return new CSP<(int, int), int>(variables, domains, neighbors, constraint);
    }

    /// <summary>
    /// Applique une solution CSP a une grille Sudoku.
    /// </summary>
    public static void ApplySolution(SudokuGrid grid, Dictionary<(int, int), int> assignment)
    {
        foreach (var ((i, j), value) in assignment)
            grid.Cells[i][j] = value;
    }
}

Console.WriteLine("SudokuCSPBuilder defini.");

## 4. Backtracking simple

L'algorithme de **backtracking** est la base de toute resolution CSP. Il explore recursivement l'espace des assignations, detectant les conflits au fur et a mesure.

In [ ]:
/// <summary>
/// Backtracking simple pour CSP.
/// Choisit les variables dans l'ordre, explore les valeurs dans l'ordre.
/// </summary>
public static class BacktrackingSimple
{
    public static Dictionary<TVariable, TValue>? Solve<TVariable, TValue>(
        CSP<TVariable, TValue> csp,
        Dictionary<TVariable, TValue>? assignment = null)
        where TVariable : notnull
    {
        assignment ??= new Dictionary<TVariable, TValue>();

        if (csp.IsComplete(assignment))
            return assignment;

        // Choisir la premiere variable non assignee (ordre naif)
        var unassigned = csp.Variables.Where(v => !assignment.ContainsKey(v)).ToList();
        var var = unassigned[0];

        foreach (var val in csp.Domains[var])
        {
            csp.NumAssignments++;
            
            if (csp.IsConsistent(var, val, assignment))
            {
                assignment[var] = val;
                var result = Solve(csp, assignment);
                if (result != null)
                    return result;
                assignment.Remove(var);
                csp.NumBacktracks++;
            }
        }

        return null; // Echec
    }
}

Console.WriteLine("BacktrackingSimple defini.");

## 5. Heuristiques : MRV et LCV

### MRV (Minimum Remaining Values)
Heuristique de **selection de variable** : choisir la variable avec le plus petit domaine restant.

### LCV (Least Constraining Value)
Heuristique d'**ordonnancement des valeurs** : essayer d'abord la valeur qui elimine le moins de possibilites chez les voisins.

In [ ]:
/// <summary>
/// Heuristiques pour la resolution CSP.
/// </summary>
public static class CSPHeuristics
{
    /// <summary>
    /// MRV : Selectionne la variable avec le moins de valeurs viables.
    /// En cas d'egalite, utilise le degre (nombre de voisins non assignes).
    /// </summary>
    public static TVariable SelectMRV<TVariable, TValue>(
        CSP<TVariable, TValue> csp,
        Dictionary<TVariable, TValue> assignment,
        Dictionary<TVariable, List<TValue>>? currentDomains = null)
        where TVariable : notnull
    {
        currentDomains ??= csp.Domains;
        
        var unassigned = csp.Variables.Where(v => !assignment.ContainsKey(v)).ToList();

        // Compter les valeurs viables pour chaque variable
        int RemainingValues(TVariable v)
        {
            return currentDomains[v].Count(val => csp.IsConsistent(v, val, assignment));
        }

        // Degre : nombre de voisins non assignes
        int Degree(TVariable v)
        {
            return csp.Neighbors[v].Count(n => !assignment.ContainsKey(n));
        }

        // MRV croissant, puis degre decroissant
        return unassigned
            .OrderBy(v => RemainingValues(v))
            .ThenByDescending(v => Degree(v))
            .First();
    }

    /// <summary>
    /// LCV : Ordonne les valeurs par nombre de conflits croissant.
    /// La valeur qui elimine le moins de possibilites est essayee en premier.
    /// </summary>
    public static IEnumerable<TValue> OrderLCV<TVariable, TValue>(
        CSP<TVariable, TValue> csp,
        TVariable var,
        Dictionary<TVariable, TValue> assignment,
        Dictionary<TVariable, List<TValue>>? currentDomains = null)
        where TVariable : notnull
    {
        currentDomains ??= csp.Domains;

        int Conflicts(TValue val)
        {
            int count = 0;
            foreach (var neighbor in csp.Neighbors[var])
            {
                if (!assignment.ContainsKey(neighbor))
                {
                    foreach (var nval in currentDomains[neighbor])
                    {
                        if (!csp.ConstraintFunc(var, val, neighbor, nval))
                            count++;
                    }
                }
            }
            return count;
        }

        return currentDomains[var].OrderBy(v => Conflicts(v));
    }
}

Console.WriteLine("Heuristiques MRV et LCV definies.");

## 6. Backtracking ameliore (MRV + LCV)

Nous combinons les deux heuristiques pour obtenir un solveur beaucoup plus efficace.

In [ ]:
/// <summary>
/// Backtracking avec heuristiques MRV et LCV.
/// </summary>
public static class BacktrackingImproved
{
    public static Dictionary<TVariable, TValue>? Solve<TVariable, TValue>(
        CSP<TVariable, TValue> csp,
        Dictionary<TVariable, TValue>? assignment = null,
        Dictionary<TVariable, List<TValue>>? currentDomains = null,
        bool useMRV = true,
        bool useLCV = true)
        where TVariable : notnull
    {
        assignment ??= new Dictionary<TVariable, TValue>();
        currentDomains ??= csp.CopyDomains();

        if (csp.IsComplete(assignment))
            return assignment;

        // Selection de variable
        var var = useMRV 
            ? CSPHeuristics.SelectMRV(csp, assignment, currentDomains)
            : csp.Variables.First(v => !assignment.ContainsKey(v));

        // Ordonnancement des valeurs
        var values = useLCV 
            ? CSPHeuristics.OrderLCV(csp, var, assignment, currentDomains)
            : currentDomains[var];

        foreach (var val in values)
        {
            csp.NumAssignments++;
            
            if (csp.IsConsistent(var, val, assignment))
            {
                assignment[var] = val;
                var result = Solve(csp, assignment, currentDomains, useMRV, useLCV);
                if (result != null)
                    return result;
                assignment.Remove(var);
                csp.NumBacktracks++;
            }
        }

        return null;
    }
}

Console.WriteLine("BacktrackingImproved defini.");

## 7. Forward Checking

Le **Forward Checking** propage l'assignation d'une variable vers ses voisins immediats, reduisant leurs domaines et detectant les echecs plus tot.

In [ ]:
/// <summary>
/// Forward Checking : propage l'assignation vers les voisins.
/// </summary>
public static class ForwardChecking
{
    /// <summary>
    /// Propage l'assignation var=val vers les voisins non assignes.
    /// Retourne la liste des valeurs retirees pour restauration.
    /// </summary>
    public static (List<(TVariable, TValue)> Removals, bool Success) Propagate<TVariable, TValue>(
        CSP<TVariable, TValue> csp,
        TVariable var,
        TValue val,
        Dictionary<TVariable, TValue> assignment,
        Dictionary<TVariable, List<TValue>> currentDomains)
        where TVariable : notnull
    {
        var removals = new List<(TVariable, TValue)>();

        foreach (var neighbor in csp.Neighbors[var])
        {
            if (!assignment.ContainsKey(neighbor))
            {
                var toRemove = new List<TValue>();
                foreach (var nval in currentDomains[neighbor])
                {
                    if (!csp.ConstraintFunc(var, val, neighbor, nval))
                    {
                        toRemove.Add(nval);
                        removals.Add((neighbor, nval));
                    }
                }

                foreach (var r in toRemove)
                    currentDomains[neighbor].Remove(r);

                if (currentDomains[neighbor].Count == 0)
                    return (removals, false); // Domaine vide = echec
            }
        }

        return (removals, true);
    }

    /// <summary>
    /// Restaure les valeurs retirees.
    /// </summary>
    public static void Restore<TVariable, TValue>(
        Dictionary<TVariable, List<TValue>> currentDomains,
        List<(TVariable, TValue)> removals)
        where TVariable : notnull
    {
        foreach (var (var, val) in removals)
            currentDomains[var].Add(val);
    }

    /// <summary>
    /// Backtracking avec Forward Checking.
    /// </summary>
    public static Dictionary<TVariable, TValue>? Solve<TVariable, TValue>(
        CSP<TVariable, TValue> csp,
        Dictionary<TVariable, TValue>? assignment = null,
        Dictionary<TVariable, List<TValue>>? currentDomains = null)
        where TVariable : notnull
    {
        assignment ??= new Dictionary<TVariable, TValue>();
        currentDomains ??= csp.CopyDomains();

        if (csp.IsComplete(assignment))
            return assignment;

        var var = CSPHeuristics.SelectMRV(csp, assignment, currentDomains);

        foreach (var val in CSPHeuristics.OrderLCV(csp, var, assignment, currentDomains))
        {
            csp.NumAssignments++;

            if (csp.IsConsistent(var, val, assignment))
            {
                assignment[var] = val;

                var (removals, success) = Propagate(csp, var, val, assignment, currentDomains);

                if (success)
                {
                    var result = Solve(csp, assignment, currentDomains);
                    if (result != null)
                        return result;
                }

                Restore(currentDomains, removals);
                assignment.Remove(var);
                csp.NumBacktracks++;
            }
        }

        return null;
    }
}

Console.WriteLine("ForwardChecking defini.");

## 8. Arc Consistency (AC-3)

L'algorithme **AC-3** (Arc Consistency Algorithm #3) assure que pour chaque arc (Xi, Xj), toute valeur de Xi a un support dans Xj. C'est une propagation plus puissante que le Forward Checking.

In [ ]:
/// <summary>
/// Algorithme AC-3 pour la consistance d'arc.
/// </summary>
public static class AC3
{
    /// <summary>
    /// Rend l'arc (xi, xj) arc-consistent.
    /// Retourne true si le domaine de xi a ete modifie.
    /// </summary>
    private static bool Revise<TVariable, TValue>(
        CSP<TVariable, TValue> csp,
        TVariable xi,
        TVariable xj,
        Dictionary<TVariable, List<TValue>> currentDomains)
        where TVariable : notnull
    {
        bool revised = false;
        var toRemove = new List<TValue>();

        foreach (var val_i in currentDomains[xi])
        {
            // Chercher un support dans xj
            bool hasSupport = currentDomains[xj]
                .Any(val_j => csp.ConstraintFunc(xi, val_i, xj, val_j));

            if (!hasSupport)
            {
                toRemove.Add(val_i);
                revised = true;
            }
        }

        foreach (var val in toRemove)
            currentDomains[xi].Remove(val);

        return revised;
    }

    /// <summary>
    /// Rend le CSP arc-consistent.
    /// Retourne false si un domaine devient vide (echec).
    /// </summary>
    public static bool Run<TVariable, TValue>(
        CSP<TVariable, TValue> csp,
        Dictionary<TVariable, List<TValue>> currentDomains,
        List<(TVariable, TVariable)>? arcs = null)
        where TVariable : notnull
    {
        var queue = new Queue<(TVariable, TVariable)>(
            arcs ?? csp.GetArcs());

        while (queue.Count > 0)
        {
            var (xi, xj) = queue.Dequeue();

            if (Revise(csp, xi, xj, currentDomains))
            {
                if (currentDomains[xi].Count == 0)
                    return false; // Domaine vide = echec

                // Ajouter les arcs (xk, xi) pour k != j
                foreach (var xk in csp.Neighbors[xi])
                {
                    if (!xk.Equals(xj))
                        queue.Enqueue((xk, xi));
                }
            }
        }

        return true;
    }
}

Console.WriteLine("AC3 defini.");

## 9. MAC (Maintaining Arc Consistency)

L'algorithme **MAC** combine le backtracking avec AC-3 : apres chaque assignation, on maintient la consistance d'arc sur tout le CSP. C'est l'algorithme le plus puissant pour les CSP difficiles.

In [ ]:
/// <summary>
/// MAC (Maintaining Arc Consistency) : Backtracking + AC-3.
/// </summary>
public static class MAC
{
    public static Dictionary<TVariable, TValue>? Solve<TVariable, TValue>(
        CSP<TVariable, TValue> csp,
        Dictionary<TVariable, TValue>? assignment = null,
        Dictionary<TVariable, List<TValue>>? currentDomains = null)
        where TVariable : notnull
    {
        assignment ??= new Dictionary<TVariable, TValue>();
        currentDomains ??= csp.CopyDomains();

        if (csp.IsComplete(assignment))
            return assignment;

        var var = CSPHeuristics.SelectMRV(csp, assignment, currentDomains);

        foreach (var val in CSPHeuristics.OrderLCV(csp, var, assignment, currentDomains))
        {
            csp.NumAssignments++;

            if (csp.IsConsistent(var, val, assignment))
            {
                assignment[var] = val;

                // Sauvegarder les domaines
                var savedDomains = currentDomains.ToDictionary(kvp => kvp.Key, kvp => kvp.Value.ToList());

                // Reduire le domaine a {val}
                currentDomains[var] = new List<TValue> { val };

                // Executer AC-3 sur les arcs affectes
                var arcs = csp.Neighbors[var]
                    .Where(n => !assignment.ContainsKey(n))
                    .Select(n => (n, var))
                    .ToList();

                bool success = AC3.Run(csp, currentDomains, arcs);

                if (success)
                {
                    var result = Solve(csp, assignment, currentDomains);
                    if (result != null)
                        return result;
                }

                // Restaurer les domaines
                foreach (var kvp in savedDomains)
                    currentDomains[kvp.Key] = kvp.Value;
                assignment.Remove(var);
                csp.NumBacktracks++;
            }
        }

        return null;
    }
}

Console.WriteLine("MAC defini.");

## 10. Solveur AIMA pour Sudoku

Nous encapsulons tous les algorithmes dans une classe `AIMASolver` implementant `ISudokuSolver`.

In [ ]:
using System.Diagnostics;

public enum CSPStrategy
{
    BacktrackingSimple,
    BacktrackingMRVLCV,
    ForwardChecking,
    MAC
}

public class AIMASolver : ISudokuSolver
{
    public CSPStrategy Strategy { get; set; } = CSPStrategy.MAC;
    public long SolveTimeMs { get; private set; }
    public int NumAssignments { get; private set; }
    public int NumBacktracks { get; private set; }

    public SudokuGrid Solve(SudokuGrid s)
    {
        var csp = SudokuCSPBuilder.BuildCSP(s);

        var stopwatch = Stopwatch.StartNew();

        Dictionary<(int, int), int>? solution = Strategy switch
        {
            CSPStrategy.BacktrackingSimple => BacktrackingSimple.Solve(csp),
            CSPStrategy.BacktrackingMRVLCV => BacktrackingImproved.Solve(csp),
            CSPStrategy.ForwardChecking => ForwardChecking.Solve(csp),
            CSPStrategy.MAC => MAC.Solve(csp),
            _ => throw new ArgumentException($"Strategie inconnue: {Strategy}")
        };

        stopwatch.Stop();
        SolveTimeMs = stopwatch.ElapsedMilliseconds;
        NumAssignments = csp.NumAssignments;
        NumBacktracks = csp.NumBacktracks;

        if (solution != null)
            SudokuCSPBuilder.ApplySolution(s, solution);

        return s;
    }
}

Console.WriteLine("AIMASolver defini.");

## 11. Test et benchmark

Comparons les quatre strategies sur des puzzles de differentes difficultes.

In [ ]:
// Chargement des puzzles
var puzzles = SudokuHelper.LoadPuzzles("Puzzles/Sudoku_Easy51.txt");
Console.WriteLine($"{puzzles.Count} puzzles charges.\n");

// Test sur un puzzle
var puzzle = puzzles[0];
Console.WriteLine("Puzzle original:");
SudokuHelper.DisplayGrid(puzzle);

In [ ]:
// Test avec MAC (le plus performant)
var solver = new AIMASolver { Strategy = CSPStrategy.MAC };
var solution = solver.Solve(puzzle.Clone());

Console.WriteLine($"\nSolution (MAC): {solver.SolveTimeMs}ms, {solver.NumAssignments} assignations, {solver.NumBacktracks} backtracks");
SudokuHelper.DisplayGrid(solution);
Console.WriteLine($"\nSolution valide: {SudokuHelper.IsValid(solution)}");

### Interpretation : resultat MAC

La solution est trouvee presque instantanement. L'algorithme MAC :
1. Utilise MRV pour choisir la cellule la plus contrainte
2. Utilise LCV pour essayer les valeurs les moins contraignantes
3. Maintient la consistance d'arc apres chaque assignation

In [ ]:
// Comparaison des strategies sur plusieurs puzzles
var strategies = new[]
{
    CSPStrategy.BacktrackingSimple,
    CSPStrategy.BacktrackingMRVLCV,
    CSPStrategy.ForwardChecking,
    CSPStrategy.MAC
};

Console.WriteLine("Benchmark sur 5 puzzles faciles");
Console.WriteLine("=" + new string('=', 75));
Console.WriteLine($"{'Strategie',-25} {'Assigns',>10} {'Backtracks',>12} {'Temps(ms)',>12} {'Succes',>8}");
Console.WriteLine("-" + new string('-', 75));

foreach (var strategy in strategies)
{
    long totalAssigns = 0, totalBacktracks = 0, totalTime = 0;
    int successes = 0;

    for (int i = 0; i < Math.Min(5, puzzles.Count); i++)
    {
        solver = new AIMASolver { Strategy = strategy };
        var sol = solver.Solve(puzzles[i].Clone());
        
        totalAssigns += solver.NumAssignments;
        totalBacktracks += solver.NumBacktracks;
        totalTime += solver.SolveTimeMs;
        if (SudokuHelper.IsValid(sol)) successes++;
    }

    Console.WriteLine($"{strategy,-25} {totalAssigns/5,10} {totalBacktracks/5,12} {totalTime,12} {successes}/5");
}

Console.WriteLine("=" + new string('=', 75));

### Interpretation : comparaison des strategies

| Strategie | Assignations | Backtracks | Analyse
|-----------|-------------|------------|--------
| **Simple** | Eleve | Eleve | Ne profite d'aucune optimisation
| **MRV+LCV** | Reduit | Reduit | Fail-first + succeed-first
| **Forward Checking** | Tres reduit | Tres reduit | Propagation 1-niveau
| **MAC** | Minimal | Minimal | Propagation complete

**Observations cles** :
1. **MRV** est l'heuristique la plus impactante (reduction souvent > 10x)
2. **FC** ajoute un gain significatif en detectant les echecs plus tot
3. **MAC** est optimal mais plus couteux par noeud (AC-3 a chaque pas)
4. Sur les puzzles faciles, la difference peut etre subtile

## 12. Exercices

### Exercice 1 : Comptage des contraintes

Combien de contraintes binaires contient le CSP du Sudoku ? Verifiez votre reponse en comptant les arcs.

<details>
<summary><b>Solution</b></summary>

Chaque cellule a 20 voisins (8 meme ligne + 8 meme colonne + 4 meme bloc hors ligne/colonne).
Il y a 81 cellules, donc 81 x 20 = 1620 arcs orientes, soit 810 contraintes binaires.

```csharp
var csp = SudokuCSPBuilder.BuildCSP(puzzles[0]);
var arcs = csp.GetArcs();
Console.WriteLine($"Nombre d'arcs: {arcs.Count}"); // 1620
Console.WriteLine($"Nombre de contraintes: {arcs.Count / 2}"); // 810
```
</details>

### Exercice 2 : AC-3 seul

AC-3 peut parfois resoudre un Sudoku completement sans backtracking. Testez sur un puzzle facile.

<details>
<summary><b>Solution</b></summary>

```csharp
var csp = SudokuCSPBuilder.BuildCSP(puzzles[0]);
var domains = csp.CopyDomains();
bool success = AC3.Run(csp, domains);

int singletons = domains.Count(kvp => kvp.Value.Count == 1);
Console.WriteLine($"AC-3 reussi: {success}");
Console.WriteLine($"Domaines singleton: {singletons}/81");

// Si tous sont singletons, AC-3 a resolu le Sudoku !
if (singletons == 81)
    Console.WriteLine("AC-3 a resolu le Sudoku sans backtracking !");
```
</details>

### Exercice 3 : Visualisation de la propagation

Modifiez le solveur MAC pour afficher la taille des domaines apres chaque assignation.

<details>
<summary><b>Indice</b></summary>

Ajoutez un parametre `verbose` a la methode `MAC.Solve` et affichez le nombre de domaines reduits apres chaque appel a AC-3.
</details>

## 13. Recapitulatif

### Algorithmes implementes

| Algorithme | Propagation | Detection d'echec | Performance
|------------|-------------|-------------------|------------
| **Backtracking** | Aucune | A l'assignation | Lente
| **BT + MRV + LCV** | Aucune | A l'assignation | Amelioree
| **Forward Checking** | 1 niveau | Domaine voisin vide | Rapide
| **MAC** | Complete (AC-3) | Domaine vide global | Optimale

### Heuristiques

| Heuristique | Role | Effet
|-------------|------|------
| **MRV** | Selection de variable | Fail-first
| **LCV** | Ordonnancement valeurs | Succeed-first
| **Degree** | Departage MRV | Priorite aux variables contraintes

### Liens avec les autres notebooks

- **Sudoku-7-Norvig** : Propagation plus poussees (naked/hidden singles)
- **Sudoku-8-HumanStrategies** : Techniques d'inference avancees
- **Sudoku-10-ORTools** : Bibliotheque industrielle avec propagation optimisee
- **Search-6/7/8** : Fondements theoriques des CSP

### References

- Russell, S. & Norvig, P. *Artificial Intelligence: A Modern Approach*, 4e ed., Chapitre 6
- Mackworth, A. K. *Consistency in Networks of Relations* (1977)
- Dechter, R. *Constraint Processing*, Cambridge University Press, 2003

---

## Navigation

| << Precedent | [Index](./README.md) | Suivant >> |
|-------------|---------------------|-----------|
| [Sudoku-5-PSO](Sudoku-5-PSO.ipynb) | | [Sudoku-7-Norvig](Sudoku-7-Norvig.ipynb) |